# Basics

Preliminary (done in previous session): import libraries 

In [31]:
import numpy as np
import seaborn as sns # seaborn
import pandas as pd

Loading data

In [32]:
beatalign_all = pd.read_csv("datasets/beatalign_vanderSteenvanVugt.txt",sep=" ")

In [37]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data[:, [0, 2]]
y = iris.target

# Machine learning in python

we will demonstrate classification methods with scikit-learn (sklearn), one the most used machine learning library in python (http://scikit-learn.org/stable/)

Sklearn API: http://scikit-learn.org/stable/modules/classes.html

**Workflow:**
1. intialise ML method
2. train it
3. test it

**Workflow in pseudo-code:**
1. `method = name()`
2. `method.fit(X_train, y_train)`
3. `pred_X = method.predict(X_test)`


# Classification

## Linear models

Example: Support Vector Machine (SVM)

In [33]:
# SVM in sklearn
from sklearn import svm
classifier = svm.SVC()

In [34]:
classifier

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [35]:
classifier = svm.SVC(kernel='linear')

In [36]:
classifier

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)